### Рекомендательная система (онлайн кинотеар).

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
# Создаем названия колонок для датафрейма films
names = ['movie id', 'movie title', 'release date', 'release year', 'IMDb URL', 'unknown', 'Action',
         'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
         'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# Подгружаем файл с перечнем фильмов
films = pd.read_csv('u.item.csv', sep='|', names=names, encoding='latin-1')
films.head()

In [ ]:
# Посмотрим типы данных
films.info()

In [ ]:
# Год отформатируем в int чуть позднее
# Создаем названия колонок для датафрейма ratings
names_2 = ['user id', 'item id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data.csv',  sep='\t', names=names_2)
ratings.head()

In [ ]:
# Найдем пользователя, который поставил больше всего оценок
ratings['user id'].value_counts().head()

In [ ]:
# Больше всего оценок поставил пользователь с id 405
# Сформируем датафрейм по пользователю 405
user_405 = ratings[ratings['user id'] == 405]
user_405.head()

In [ ]:
# Проверим наш датафрейм на пропуски
user_405.info()

In [ ]:
# Добавим к нашему датафрейму по пользователю 405 жанры
merge = user_405.merge(films, left_on='item id', right_on='movie id')
merge.head()

In [ ]:
# В дальнейшем нам потребуется год выпуска фильма, поэтому приведем колонку release date к формату даты, 
# затем на основании данной колонки заполним колонку release year. она как раз у нас пустая
merge['release date'] = pd.to_datetime(merge['release date'],  format='%d-%b-%Y')
merge['release year'] = merge['release date'].dt.year
merge.head()

In [ ]:
merge.info()

In [ ]:
# На данном этапе сформируем датафрейм с общим количеством оценок от всех пользователей на фильм 
# и суммарной оценкой от всех пользователей.
ratings_2 = ratings.groupby('item id').sum()[['rating']].reset_index()
ratings_3 = ratings.groupby('item id').count()[['rating']].reset_index()
merge_2 = ratings_2 .merge(ratings_3, left_on='item id', right_on='item id')
merge_2.columns = ['movie_id', 'sum_rating', 'count_rating']
merge_2.sort_values('count_rating', ascending=False)

In [ ]:
# Соединим датафреймы
merge_3 = merge.merge(merge_2, left_on='movie id', right_on='movie_id')
merge_3.head()

In [ ]:
# Выведем названия колонок для удобства
merge_3.columns

In [ ]:
# Сформируем итоговый датафрейм для нашей рекомендательной модели оставив необходиимые признаки
final_df = merge_3[['release year', 'unknown', 'Action',
       'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 
        'count_rating',  'sum_rating', 'rating']]

final_df.head()

In [ ]:
# Начинаем строить модель
# Импортируем нужные нам библиотеки
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
final_df.columns

In [ ]:
# Отделим колонки с признаками от колонки с целевой переменной (которую прогнозируем)
# это нужно для корректной работы sklearn

In [ ]:
X = final_df[['release year', 'unknown', 'Action', 'Adventure', 'Animation', 
              'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
              'Thriller', 'War', 'Western', 'count_rating', 'sum_rating']]

y = final_df['rating']

In [ ]:
X

In [ ]:
y

In [ ]:
# Разделим данные на часть для обучения и часть для проверки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
# Приведем данные к одному масштабу
sc = StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Делаем модель
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.coef_

In [ ]:
# Оценим работу модели

In [ ]:
# Оценим модель при помощи MSE

In [ ]:
mean_squared_error(y_test, lr.predict(X_test))

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

In [ ]:
# Оценим модель при помощи MАE

In [ ]:
mean_absolute_error(y_test, lr.predict(X_test))

In [ ]:
mean_absolute_error(y_train, lr.predict(X_train))